In [ ]:
!pip install torchvision

In [1]:

import numpy as np
import autokeras as ak
from keras.preprocessing.image import ImageDataGenerator
from torchvision.transforms import transforms
from torchvision.transforms.functional import to_tensor
from torchvision.transforms import v2

In [2]:
data_dir = 'Dog Breed Dataset Reduced'

# Use ImageDataGenerator to load and preprocess the images
datagen = ImageDataGenerator(rescale = 1. / 255)

# Define the data generators for training, validation, and testing
train_generator = datagen.flow_from_directory(
    directory = f'{data_dir}/train',
    target_size = (256, 256),
    batch_size = 16,
    class_mode = 'categorical'
)


test_generator = datagen.flow_from_directory(
    directory = f'{data_dir}/test',
    target_size = (256, 256),
    batch_size = 1,
    class_mode = 'categorical',
    shuffle = False
)

Found 1443 images belonging to 29 classes.
Found 374 images belonging to 29 classes.


In [3]:
# Define the AutoAugment policy
autoaugment_policy = v2.AutoAugmentPolicy.IMAGENET
autoaugment_transform = v2.AutoAugment(autoaugment_policy)

# Initialize empty lists for training and test data and labels
x_train, y_train = [], []
x_test, y_test = [], []
test = []
test1 = []

# Load training data in batches
for _ in range(len(train_generator)):
    batch_x, batch_y = train_generator.next()

    # Apply AutoAugment to each image in the batch
    augmented_images = []
    augmented_images_labels = []
    n = 0
    for image in batch_x:
        # Convert the image to a tensor (required for AutoAugment)
        tensor_image = to_tensor(image)
        
        # plt.imshow(image)
        # plt.title('Original Image')
        # plt.show()
        # Apply AutoAugment policy
        for _ in range(3):  
            augmented_tensor = autoaugment_transform(tensor_image)
            augmented_image = transforms.ToPILImage()(augmented_tensor)
            augmented_images.append(np.array(augmented_image))
            augmented_images_labels.append(np.array(batch_y[n]))
            # plt.imshow(np.array(augmented_image))
            # plt.title('Augmented Image')
            # plt.show()

        n += 1

    x_train.append(batch_x)
    y_train.append(batch_y)
    x_train.append(augmented_images)
    y_train.append(augmented_images_labels)

# Load test data in batches
for _ in range(len(test_generator)):
    batch_x, batch_y = test_generator.next()

    # Apply AutoAugment to each image in the batch
    augmented_images = []
    augmented_images_labels = []
    n = 0
    for image in batch_x:
        # Convert the image to a tensor (required for AutoAugment)
        tensor_image = to_tensor(image)
        
        # plt.imshow(image)
        # plt.title('Original Image')
        # plt.show()
        # Apply AutoAugment policy
        for _ in range(3):
            augmented_tensor = autoaugment_transform(tensor_image)
            augmented_image = transforms.ToPILImage()(augmented_tensor)
            augmented_images.append(np.array(augmented_image))
            augmented_images_labels.append(np.array(batch_y[n]))
            # plt.imshow(np.array(augmented_image))
            # plt.title('Augmented Image')
            # plt.show()

        n += 1

    
    x_test.append(batch_x)
    y_test.append(batch_y)
    x_test.append(augmented_images)
    y_test.append(augmented_images_labels)


In [4]:
# Concatenate the training and test data
x_train = np.concatenate(x_train)
x_test = np.concatenate(x_test)

# Concatenate the labels and then convert to integer labels
y_train = np.concatenate(y_train)
y_train = np.argmax(y_train, axis=1)

y_test = np.concatenate(y_test)
y_test = np.argmax(y_test, axis=1)


In [5]:
clf = ak.ImageClassifier(
    max_trials = 1,
    objective = 'val_accuracy',
    overwrite = True,
)

# Search for the best model architecture
clf.fit(x_train, y_train, epochs = 10)





Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
vanilla           |vanilla           |image_block_1/block_type
True              |True              |image_block_1/normalize
False             |False             |image_block_1/augment
3                 |3                 |image_block_1/conv_block_1/kernel_size
1                 |1                 |image_block_1/conv_block_1/num_blocks
2                 |2                 |image_block_1/conv_block_1/num_layers
True              |True              |image_block_1/conv_block_1/max_pooling
False             |False             |image_block_1/conv_block_1/separable
0.25              |0.25              |image_block_1/conv_block_1/dropout
32                |32                |image_block_1/conv_block_1/filters_0_0
64                |64                |image_block_1/conv_block_1/filters_0_1
flatten           |flatten           |classification_head_1/spatial_reduction_1/reduction_type
0.5               |0.5       

KeyboardInterrupt: 

In [8]:
results = clf.evaluate(x_test, y_test)
print(results)
print("Test accuracy:", results[1])

2/2 [==============================] - 2s 631ms/step - loss: 1.1038e-08 - accuracy: 1.0000
[1.1037893266063747e-08, 1.0]
Test accuracy: 1.0


In [9]:
from sklearn.metrics import classification_report

In [10]:
y_pred = clf.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate precision, recall, accuracy, and F1 score
report = classification_report(y_test, y_pred_classes)
print(report)

2/2 [==============================] - 2s 867ms/step
              precision    recall  f1-score   support

           0       0.33      1.00      0.50        18
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00        18

    accuracy                           0.33        54
   macro avg       0.11      0.33      0.17        54
weighted avg       0.11      0.33      0.17        54



c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [ ]:

clf.export_model().save("Dog Breed Model")